## Test Create CHCs

In this file we test the creation of CHCs from a given program.

First we import the necessary libraries and set up the Z3 solver.

In [1]:
from z3 import And, Ints, Int, Array, IntSort, ForAll, Implies, Or
import z3
import graphviz
z3.set_param(proof=True)
from sw.verify.stasm_to_chcs import create_chcs

# Sanity test
Now let's create a simple program and generate the CHCs for it.
This program is taken straight out of the class presentation.
It is a sanity test case for our implementation.
All it does is `stack[sp-2] + stack[sp-1] * 13`.  

In [2]:
program = [("PUSH", 13),
              ("POP", 2),
              ("ALU", "MUL"),
              ("POP", 2),
              ("ALU", "ADD"),
              ("POP", 1)]

In our case we are going to set 
`stack[0] = 1, stack[1] = 5`
and test that the output is `1 + 5 * 13`

In [3]:

stack = Array("stack", IntSort(), IntSort())
sp, r0, r1 = Ints('sp r0 r1')
a, b = Ints('a b')
chcs = create_chcs(pre_condition=And(stack[0] == a, stack[1] == b, sp == 2),
             input_vars=[a,b],
             program=program,
             post_condition=stack[sp] ==  a + b * 13)
d = {'xform.inline_eager': False, 'xform.inline_linear': False}
chcs.solve()

version 7


# Find max element in array
Now lets test the find max program from milestone 0
First, import some macros:

In [4]:
def READ_FROM_ARRAY():
    """
    Pops the top as index and then base addr and push the value at that index from the array at addr
    :return:
    """
    return [
        ("POP", 2),  # r0 = index, r1 = base addr
        ("ALU", "ADD"),  # push base addr + index
        ("POP", 1),   # r0 = base addr + index
        ("LOAD", 0),  # push value at base addr + index
    ]

Now writing the program:

In [5]:
program_find_max = [
    ("PUSH", 0),
    ("DUP", 2),
    *READ_FROM_ARRAY(),
    # Stack is now: [arr_addr, length, a[0]]
    # Now let's define max = a[0]
    ("DUP", 0),
    # Stack is now: [arr_addr, length, a[0], mx=a[0]]
    # Now after we saved a[0] on stack we can use this mem for i
    # lets set i = 1 to memory address &a[0]
    ("PUSH", 1),
    ("DUP", 4),
    ("POP", 2),
    ("STOR", 0),
    "CHECK_COND:",
    # First put i in r0 and n in r1
    ("DUP", 3),
    ("POP", 1),
    ("LOAD", 0),
    ("DUP", 3),
    ("POP", 2),
    # Check i < n
    ("ALU", "LT"),
    ("POP", 1),
    ("JNZ", "LOOP_BODY"),
    ("JMP", "END"),
    "LOOP_BODY:",
    # read i from mem and put on stack
    ("DUP", 3),
    ("POP", 1),
    ("LOAD", 0),
    # put base addr on stack
    ("DUP", 4),
    # Put a[i] on stack
    *READ_FROM_ARRAY(),
    # Put mx on stack
    ("DUP", 1),
    # now stack is [base_addr, n, a[0], mx, a[i], mx]
    ("POP", 2),
    ("ALU", "LT"),
    # now stack is [base_addr, n, a[0], mx, (result a[i] < mx)]
    ("POP", 1),
    ("JNZ", "INC_I"),
    ("JMP", "UPDATE"),
    "INC_I:",
    # Put i on stack
    ("PUSH", 0),
    ("DUP", 4),
    *READ_FROM_ARRAY(),
    ("PUSH", 1),
    ("POP", 2),
    ("ALU", "ADD"),
    ("DUP", 4),
    ("POP", 2),
    ("STOR", 0),
    ("JMP", "CHECK_COND"),
    "UPDATE:",
    ("POP", 1),  # remove old mx from stack
    # Push base addr
    ("DUP", 2),
    ("POP", 1),
    ("LOAD", 0),
    # Now i is on top of stack
    ("DUP", 3),
    *READ_FROM_ARRAY(),  # so now a[i] is where mx was on stack so mx = a[i]
    ("JMP", "INC_I"),
    "END:",
    # Restore a[0]:
    # Put a[0] on stack (we saved it before):
    ("DUP", 1),
    # Put base_addr on stack:
    ("DUP", 4),
    ("POP", 2),
    ("STOR", 0),
    ("POP", 2),
    ("POP", 1),
    ("POP", 1)
]

Now creating the verification (We are testing array of size 1: `[2]` and making sure the return value is 2 (`r1==2`)

In [6]:
stack = Array("stack", IntSort(), IntSort())
memory = Array("memory", IntSort(), IntSort())
sp, r0, r1 = Ints('sp r0 r1')

pre_condition = And(
    stack[0] == 0, stack[1] == 0, stack[2] == 1, sp == 2,
    memory[0] == 2
)
chcs = create_chcs(pre_condition=pre_condition,
                   input_vars=[],
                   program=program_find_max,
                   post_condition=And(sp == 0,
                                      memory[0] == 2,
                                      r1 == 2))
chcs.solve(d)

version 7


U31,False
U19,(¬(ν0[0] ≤ 1) ∨ ¬(ν1[-3 + ν2] ≤ -1)) ∧ ¬(ν1[ν2] ≥ 3) ∧ ¬(ν1[ν2] ≤ 1) ∧ ¬(ν2 ≥ 5) ∧ ¬(ν2 ≤ 3) ∧ (¬(ν0[0] ≥ 3) ∨ ¬(ν1[-3 + ν2] ≤ -1)) ∧ (¬(ν0[0] ≥ 3) ∨ ¬(ν1[-3 + ν2] ≥ 1)) ∧ ¬(ν1[-1 + ν2] ≥ 3) ∧ ¬(ν1[-1 + ν2] ≤ 1) ∧ (¬(ν0[0] ≤ 1) ∨ ¬(ν1[-3 + ν2] ≥ 1))
U16,¬(ν3 + -1·ν4 ≤ -1) ∧ ¬(ν1[ν2] ≥ 3) ∧ ¬(ν1[ν2] ≤ 1) ∧ ¬(ν2 ≥ 5) ∧ ¬(ν2 ≤ 3) ∧ (¬(ν0[0] ≥ 3) ∨ ¬(ν1[-3 + ν2] ≥ 1)) ∧ (¬(ν0[0] ≥ 3) ∨ ¬(ν1[-3 + ν2] ≤ -1)) ∧ ¬(ν1[-1 + ν2] ≥ 3) ∧ ¬(ν1[-1 + ν2] ≤ 1) ∧ (¬(ν0[0] ≤ 1) ∨ ¬(ν1[-3 + ν2] ≥ 1)) ∧ (¬(ν0[0] ≤ 1) ∨ ¬(ν1[-3 + ν2] ≤ -1))
U37,False
U1,¬(ν1[-1 + ν2] ≥ 2) ∧ ¬(ν0[ν1[ν2] + ν1[-2 + ν2]] ≥ 3) ∧ ¬(ν0[ν1[ν2] + ν1[-2 + ν2]] ≤ 1) ∧ ¬(ν2 ≥ 4) ∧ ¬(ν2 ≤ 2) ∧ ¬(ν0[0] ≥ 3) ∧ ¬(ν0[0] ≤ 1)
U40,False
U21,False
U38,False
U42,False
U8,¬(ν1[-3 + ν2] + -1·ν1[ν2] ≥ 1) ∧ ¬(ν1[-1 + ν2] ≥ 3) ∧ ¬(ν1[-1 + ν2] ≤ 1) ∧ ¬(ν2 ≥ 6) ∧ ¬(ν2 ≤ 4) ∧ ¬(ν0[0] ≥ 3) ∧ ¬(ν1[-2 + ν2] ≥ 3) ∧ ¬(ν1[-2 + ν2] ≤ 1) ∧ ¬(ν0[0] ≤ 1) ∧ ¬(ν1[ν2] ≤ 0)
U26,False


Now lets test that it finds model for `r1==9999` (the max is 2 and not 9999, so we should get a digraph for a counterexample)

In [7]:
stack = Array("stack", IntSort(), IntSort())
memory = Array("memory", IntSort(), IntSort())
sp, r0, r1 = Ints('sp r0 r1')


pre_condition = And(
    stack[0] == 0, stack[1] == 1, sp == 2,
    memory[0] == 2
)
chcs = create_chcs(pre_condition=pre_condition,
                   input_vars=[],
                   program=program_find_max,
                   post_condition=And(sp == 0,
                                      memory[0] == 2,
                                      r1 == 9999))
chcs.solve(d)

version 7


Now lets test an array `[2,8,10,3]` of size 4 to make sure that we get SAT for `r1==10`

In [8]:
stack = Array("stack", IntSort(), IntSort())
memory = Array("memory", IntSort(), IntSort())
sp, r0, r1 = Ints('sp r0 r1')

pre_condition = And(
    stack[0] == 0, stack[1] == 0, stack[2] == 4, sp == 2,
    memory[0] == 2, memory[1] == 8, memory[2] == 10, memory[3] == 3
)
# Test correct maximum
chcs_correct = create_chcs(pre_condition=pre_condition,
                           input_vars=[],
                           program=program_find_max,
                           post_condition=And(sp == 0,
                                              memory[0] == 2, memory[1] == 8,
                                              memory[2] == 10, memory[3] == 3,
                                              r1 == 10))
chcs_correct.solve(d)

version 7


[U31 = [else ->
        And(Or(Not(Var(1)[Var(2)] <= 0),
               Not(Var(1)[-3 + Var(2)] +
                   -1*Var(0)[Var(1)[-4 + Var(2)]] >=
                   3),
               Not(Var(0)[1 +
                     Var(1)[-4 + Var(2)] +
                     Var(0)[Var(1)[-4 + Var(2)]]] <=
                   9)),
            Not(Var(1)[Var(2)] <= -1),
            Not(Var(1)[-1 + Var(2)] >= 11),
            Not(Var(2) >= 6),
            Not(Var(0)[Var(1)[-4 + Var(2)]] <= -1),
            Not(Var(1)[-4 + Var(2)] >= 1),
            Or(Not(Var(1)[-3 + Var(2)] +
                   -1*Var(0)[Var(1)[-4 + Var(2)]] <=
                   2),
               Not(Var(0)[Var(1)[-4 + Var(2)] +
                     Var(0)[Var(1)[-4 + Var(2)]]] >=
                   11)),
            Not(Var(1)[-3 + Var(2)] <= 1),
            Not(Var(0)[1] <= 7),
            Or(Not(Var(1)[-1 + Var(2)] <= 9),
               Not(Var(1)[-3 + Var(2)] +
                   -1*Var(0)[Var(1)[-4 + Var(2)]] <=
                   1)),
            Not(Var(2) <= 4),
            Not(Var(0)[1] >= 9),
            Not(Var(0)[2] <= 9),
            Not(Var(1)[-2 + Var(2)] >= 3),
            Or(Not(Var(0)[3] >= 4),
               Not(Var(1)[-4 + Var(2)] <= 2)),
            Or(Not(Var(0)[3] <= 2),
               Not(Var(1)[-4 + Var(2)] <= 2)),
            Not(Var(1)[-4 + Var(2)] <= -1),
            Not(Var(0)[2] >= 11),
            Not(Var(1)[-2 + Var(2)] <= 1),
            Or(Not(Var(1)[Var(2)] <= 0),
               Not(Var(1)[-3 + Var(2)] +
                   -1*Var(0)[Var(1)[-4 + Var(2)]] <=
                   2),
               Not(Var(0)[Var(1)[-4 + Var(2)] +
                     Var(0)[Var(1)[-4 + Var(2)]]] <=
                   9)),
            Not(Var(1)[-3 + Var(2)] +
                -1*Var(0)[Var(1)[-4 + Var(2)]] >=
                4),
            Or(Not(Var(1)[-3 + Var(2)] +
                   -1*Var(0)[Var(1)[-4 + Var(2)]] >=
                   3),
               Not(Var(0)[1 +
                     Var(1)[-4 + Var(2)] +
                     Var(0)[Var(1)[-4 + Var(2)]]] +
                   -1*
                   Var(0)[Var(1)[-4 + Var(2)] +
                     Var(0)[Var(1)[-4 + Var(2)]]] <=
                   -1)),
            Or(Not(Var(1)[-3 + Var(2)] +
                   -1*Var(0)[Var(1)[-4 + Var(2)]] >=
                   3),
               Not(Var(0)[2 +
                     Var(1)[-4 + Var(2)] +
                     Var(0)[Var(1)[-4 + Var(2)]]] >=
                   11)),
            Or(Not(Var(1)[-3 + Var(2)] +
                   -1*Var(0)[Var(1)[-4 + Var(2)]] >=
                   2),
               Not(Var(0)[1 +
                     Var(1)[-4 + Var(2)] +
                     Var(0)[Var(1)[-4 + Var(2)]]] >=
                   11)),
            Or(Not(Var(1)[Var(2)] >= 1),
               Not(Var(1)[-1 + Var(2)] <= 9)))],
 U19 = [else ->
        And(Not(Var(1)[-1 + Var(2)] <= 1),
            Not(Var(1)[Var(2)] <= 9),
            Not(Var(2) >= 5),
            Not(Var(1)[Var(2)] >= 11),
            Not(Var(0)[1] <= 7),
            Not(Var(0)[1] >= 9),
            Not(Var(2) == 3),
            Not(Var(0)[2] <= 9),
            Or(Not(Var(0)[3] >= 4),
               Not(Var(1)[-3 + Var(2)] <= 2)),
            Or(Not(Var(0)[3] <= 2),
               Not(Var(1)[-3 + Var(2)] <= 2)),
            Or(Not(Var(1)[-3 + Var(2)] == 0),
               Not(Var(1)[-1 + Var(2)] >= 3)),
            Not(Var(1)[-3 + Var(2)] <= -1),
            Not(Var(1)[-3 + Var(2)] >= 1),
            Not(Var(0)[2] >= 11))],
 U16 = [else ->
        And(Or(Not(Var(1)[Var(2)] +
                   -1*
                   Var(0)[1 +
                     Var(1)[-3 + Var(2)] +
                     Var(0)[Var(1)[-3 + Var(2)]]] <=
                   0),
               Not(Var(1)[-2 + Var(2)] +
                   -1*Var(0)[Var(1)[-3 + Var(2)]] >=
                   3),
               Not(Var(0)[1 +
                     Var(1)[-3 + Var(2)] +
                     Var(0)[Var(1)[-3 + Var(2)]]] <=
       

# Test element exists in array program
Lets test the other program from milestone 

In [9]:
program_find_v = [
    ("PUSH", 0x0),  # int i = 0
    "CHECK_COND:",
    ("DUP", 0),  # Push i on stack
    ("DUP", 3),  # Push n on stack
    ("POP", 2),  # r0 = i, r1 = n
    ("ALU", "LT"),  # Compare i < n
    ("POP", 1),  # r0 = comparison result
    ("JNZ", "LOOP_BODY"),  # If i < n, jump to loop body
    ("JMP", "NOT_FOUND"),  # Otherwise, exit loop
    "INC_I:",  # Increment i
    ("PUSH", 1),  # Push constant 1
    ("POP", 2),  # r0 = i r1 = 1
    ("ALU", "ADD"),  # Push i + 1
    ("JMP", "CHECK_COND"),  # Jump to check condition
    "LOOP_BODY:",
    ("DUP", 0),  # Push i on stack
    ("DUP", 4),  # Push base_addr on stack
    *READ_FROM_ARRAY(),  # Push a[i]
    ("DUP", 2),
    ("POP", 2),
    ("ALU", "SUB"),
    ("POP", 1),  # r0 = a[i] - v
    ("JZ", "FOUND"),  # If a[i] == v, jump to found
    ("JMP", "INC_I"),  # Otherwise, check condition
    "FOUND:",  # Found the value
    ("POP", 2),  # Emptying the stack
    ("POP", 2),
    ("PUSH", 1),  # Push True
    ("PUSH", 1),  # Push True
    ("JMP", "END"),  # Jump to end
    "NOT_FOUND:",  # Not found
    ("POP", 2),  # Emptying the stack
    ("POP", 2),
    ("PUSH", 0),  # Push False
    ("PUSH", 0),  # Push False
    "END:",
    ("POP", 2)
]

Setting up the test

In [10]:
stack = Array("stack", IntSort(), IntSort())
memory = Array("memory", IntSort(), IntSort())
sp, r0, r1 = Ints('sp r0 r1')


k = Int("k")
pre_condition = And(
    stack[0] == 0, stack[1] == 0, stack[2] == 4, stack[3] == 8,
    sp == 3,
    memory[0] == 2, memory[1] == 8, memory[2] == 1, memory[3] == 3,
    # Stack beyond sp is 0

)

# Test correct maximum
chcs_correct = create_chcs(pre_condition=pre_condition,
                           input_vars=[],
                           program=program_find_v,
                           post_condition=And(sp == 0,
                                              memory[0] == 2, memory[1] == 8, memory[2] == 1, memory[3] == 3,
                                              r1 == 1))
d = {'xform.inline_eager': False, 'xform.inline_linear': False}
chcs_correct.solve(d)

version 7


U31,False
U17,¬(ν0[3] ≥ 4) ∧ ¬(ν0[3] ≤ 2) ∧ (¬(ν1[-1 + ν2] + -1·ν0[ν3] ≥ 1) ∨ ¬(ν1[-1 + ν2] + -1·ν0[1 + ν1[-3 + ν2] + ν1[ν2]] ≥ 1)) ∧ ¬(ν1[-1 + ν2] + -1·ν0[ν3] ≤ -1) ∧ ¬(ν0[1] ≤ 7) ∧ ¬(ν1[-2 + ν2] + -1·ν1[ν2] ≤ 2) ∧ ¬(ν0[2] ≥ 2) ∧ ¬(ν2 ≤ 3) ∧ ¬(ν0[1] ≥ 9) ∧ (¬(ν1[-2 + ν2] + -1·ν1[ν2] ≤ 3) ∨ ¬(ν1[-1 + ν2] + -1·ν0[ν3] ≥ 1)) ∧ (¬(ν1[-1 + ν2] + -1·ν0[ν3] ≥ 1) ∨ ¬(ν1[-1 + ν2] + -1·ν0[1 + ν1[-3 + ν2] + ν1[ν2]] ≤ -1)) ∧ ¬(ν0[2] ≤ 0) ∧ ¬(ν0[0] ≤ 1) ∧ ¬(ν2 ≥ 5) ∧ ¬(ν0[0] ≥ 3)
U19,¬(ν0[2] ≥ 2) ∧ (¬(ν1[-1 + ν2] + -1·ν1[ν2] ≤ -1) ∨ ¬(ν1[-3 + ν2] + -1·ν0[1 + ν1[-2 + ν2] + ν1[-5 + ν2]] ≥ 1)) ∧ ¬(ν0[0] ≥ 3) ∧ ¬(ν1[-4 + ν2] + -1·ν1[-2 + ν2] ≤ 2) ∧ ¬(ν2 ≤ 5) ∧ ¬(ν2 ≥ 7) ∧ ¬(ν0[1] ≥ 9) ∧ ¬(ν0[3] ≤ 2) ∧ (¬(ν1[-1 + ν2] + -1·ν1[ν2] ≤ -1) ∨ ¬(ν1[-4 + ν2] + -1·ν1[-2 + ν2] ≤ 3)) ∧ ¬(ν0[1] ≤ 7) ∧ ¬(ν0[0] ≤ 1) ∧ ¬(ν1[-1 + ν2] + -1·ν1[ν2] ≥ 1) ∧ (¬(ν1[-1 + ν2] + -1·ν1[ν2] ≤ -1) ∨ ¬(ν1[-3 + ν2] + -1·ν0[1 + ν1[-2 + ν2] + ν1[-5 + ν2]] ≤ -1)) ∧ ¬(ν0[2] ≤ 0) ∧ ¬(ν0[3] ≥ 4)
U16,¬(ν0[2] ≥ 2) ∧ (¬(ν1[-3 + ν2] + -1·ν1[-1 + ν2] ≤ 3) ∨ ¬(ν1[-2 + ν2] + -1·ν0[ν1[ν2]] ≥ 1)) ∧ ¬(ν2 ≤ 4) ∧ ¬(ν0[2] ≤ 0) ∧ ¬(ν1[-3 + ν2] + -1·ν1[-1 + ν2] ≤ 2) ∧ ¬(ν0[3] ≤ 2) ∧ ¬(ν0[1] ≤ 7) ∧ ¬(ν2 ≥ 6) ∧ (¬(ν1[-2 + ν2] + -1·ν0[ν1[ν2]] ≥ 1) ∨ ¬(ν1[-2 + ν2] + -1·ν0[1 + ν1[-4 + ν2] + ν1[-1 + ν2]] ≤ -1)) ∧ ¬(ν1[-2 + ν2] + -1·ν0[ν1[ν2]] ≤ -1) ∧ ¬(ν0[1] ≥ 9) ∧ (¬(ν1[-2 + ν2] + -1·ν0[1 + ν1[-4 + ν2] + ν1[-1 + ν2]] ≥ 1) ∨ ¬(ν1[-2 + ν2] + -1·ν0[ν1[ν2]] ≥ 1)) ∧ ¬(ν0[0] ≥ 3) ∧ ¬(ν0[3] ≥ 4) ∧ ¬(ν0[0] ≤ 1)
U13,¬(ν1[-1 + ν2] + -1·ν1[-3 + ν2] ≥ -2) ∧ ¬(ν2 ≤ 4) ∧ ¬(ν0[3] ≥ 4) ∧ ¬(ν0[1] ≥ 9) ∧ ¬(ν0[2] ≥ 2) ∧ ¬(ν1[-2 + ν2] + -1·ν0[ν1[ν2] + ν1[-4 + ν2]] ≤ -1) ∧ ¬(ν0[3] ≤ 2) ∧ (¬(ν1[-1 + ν2] + -1·ν1[-3 + ν2] ≥ -3) ∨ ¬(ν1[-2 + ν2] + -1·ν0[ν1[ν2] + ν1[-4 + ν2]] ≥ 1)) ∧ ¬(ν2 ≥ 6) ∧ ¬(ν0[0] ≤ 1) ∧ (¬(ν0[1 + ν1[-1 + ν2] + ν1[-4 + ν2]] + -1·ν1[-2 + ν2] ≤ -1) ∨ ¬(ν1[-2 + ν2] + -1·ν0[ν1[ν2] + ν1[-4 + ν2]] ≥ 1)) ∧ (¬(ν1[-2 + ν2] + -1·ν0[ν1[ν2] + ν1[-4 + ν2]] ≥ 1) ∨ ¬(ν0[1 + ν1[-1 + ν2] + ν1[-4 + ν2]] + -1·ν1[-2 + ν2] ≥ 1)) ∧ ¬(ν0[1] ≤ 7) ∧ ¬(ν0[0] ≥ 3) ∧ ¬(ν0[2] ≤ 0)
U5,¬(ν0[1] ≤ 7) ∧ ¬(ν0[3] ≤ 2) ∧ ¬(ν0[0] ≥ 3) ∧ ¬(ν1[-1 + ν2] + -1·ν1[-3 + ν2] ≥ -2) ∧ ¬(ν0[0] ≤ 1) ∧ ¬(ν0[1] ≥ 9) ∧ ¬(ν2 ≥ 6) ∧ ¬(ν0[2] ≤ 0) ∧ (¬(ν0[ν1[-1 + ν2] + ν1[-4 + ν2]] + -1·ν1[-2 + ν2] ≤ -1) ∨ ¬(ν1[-2 + ν2] + -1·ν0[1 + ν1[-1 + ν2] + ν1[-4 + ν2]] ≤ -1)) ∧ ¬(ν0[ν1[-1 + ν2] + ν1[-4 + ν2]] + -1·ν1[-2 + ν2] ≥ 1) ∧ ¬(ν0[3] ≥ 4) ∧ ¬(ν1[ν2] ≤ 0) ∧ ¬(ν0[2] ≥ 2) ∧ (¬(ν0[ν1[-1 + ν2] + ν1[-4 + ν2]] + -1·ν1[-2 + ν2] ≤ -1) ∨ ¬(ν1[-2 + ν2] + -1·ν0[1 + ν1[-1 + ν2] + ν1[-4 + ν2]] ≥ 1)) ∧ ¬(ν2 ≤ 4) ∧ (¬(ν1[-1 + ν2] + -1·ν1[-3 + ν2] ≥ -3) ∨ ¬(ν0[ν1[-1 + ν2] + ν1[-4 + ν2]] + -1·ν1[-2 + ν2] ≤ -1))
U1,(¬(ν1[-2 + ν2] + -1·ν1[ν2] ≤ 3) ∨ ¬(ν1[-1 + ν2] + -1·ν0[ν1[ν2] + ν1[-3 + ν2]] ≥ 1)) ∧ ¬(ν1[-1 + ν2] + -1·ν0[ν1[ν2] + ν1[-3 + ν2]] ≤ -1) ∧ ¬(ν2 ≤ 3) ∧ ¬(ν0[1] ≥ 9) ∧ ¬(ν1[-2 + ν2] + -1·ν1[ν2] ≤ 2) ∧ ¬(ν0[2] ≥ 2) ∧ (¬(ν0[1 + ν1[ν2] + ν1[-3 + ν2]] + -1·ν1[-1 + ν2] ≥ 1) ∨ ¬(ν1[-1 + ν2] + -1·ν0[ν1[ν2] + ν1[-3 + ν2]] ≥ 1)) ∧ ¬(ν0[0] ≤ 1) ∧ (¬(ν0[1 + ν1[ν2] + ν1[-3 + ν2]] + -1·ν1[-1 + ν2] ≤ -1) ∨ ¬(ν1[-1 + ν2] + -1·ν0[ν1[ν2] + ν1[-3 + ν2]] ≥ 1)) ∧ ¬(ν0[3] ≥ 4) ∧ ¬(ν0[3] ≤ 2) ∧ ¬(ν0[1] ≤ 7) ∧ ¬(ν0[2] ≤ 0) ∧ ¬(ν0[0] ≥ 3) ∧ ¬(ν2 ≥ 5)
U18,¬(ν0[0] ≤ 1) ∧ ¬(ν0[2] ≤ 0) ∧ ¬(ν0[0] ≥ 3) ∧ ¬(ν0[3] ≤ 2) ∧ ¬(ν0[3] ≥ 4) ∧ (¬(ν1[-2 + ν2] + -1·ν1[ν2] ≥ 1) ∨ ¬(ν0[1 + ν1[-4 + ν2] + ν1[-1 + ν2]] + -1·ν1[-2 + ν2] ≤ -1)) ∧ ¬(ν0[2] ≥ 2) ∧ ¬(ν1[-2 + ν2] + -1·ν1[ν2] ≤ -1) ∧ (¬(ν1[-2 + ν2] + -1·ν1[ν2] ≥ 1) ∨ ¬(ν0[1 + ν1[-4 + ν2] + ν1[-1 + ν2]] + -1·ν1[-2 + ν2] ≥ 1)) ∧ ¬(ν2 ≤ 4) ∧ ¬(ν0[1] ≥ 9) ∧ ¬(ν1[-3 + ν2] + -1·ν1[-1 + ν2] ≤ 2) ∧ (¬(ν1[-2 + ν2] + -1·ν1[ν2] ≥ 1) ∨ ¬(ν1[-3 + ν2] + -1·ν1[-1 + ν2] ≤ 3)) ∧ ¬(ν2 ≥ 6) ∧ ¬(ν0[1] ≤ 7)
U21,¬(ν0[1] ≥ 9) ∧ (¬(ν1[ν2] ≤ -1) ∨ ¬(ν1[-3 + ν2] + -1·ν1[-1 + ν2] ≤ 3)) ∧ ¬(ν2 ≥ 6) ∧ ¬(ν1[-3 + ν2] + -1·ν1[-1 + ν2] ≤ 2) ∧ ¬(ν0[2] ≤ 0) ∧ ¬(ν0[3] ≤ 2) ∧ (¬(ν1[ν2] ≤ -1) ∨ ¬(ν0[1 + ν1[-1 + ν2] + ν1[-4 + ν2]] + -1·ν1[-2 + ν2] ≤ -1)) ∧ ¬(ν1[ν2] ≥ 1) ∧ ¬(ν0[0] ≤ 1) ∧ (¬(ν1[ν2] ≤ -1) ∨ ¬(ν0[1 + ν1[-1 + ν2] + ν1[-4 + ν2]] + -1·ν1[-2 + ν2] ≥ 1)) ∧ ¬(ν0[2] ≥ 2) ∧ ¬(ν0[3] ≥ 4) ∧ ¬(ν2 ≤

# Simple Test

In [11]:

program = [
    ("PUSH", 4),
    ("PUSH", 1),
    # stack is now [20,i<-1]
    "START1:",
    ("DUP", 1), # stack is now [20,i,20]
    ("DUP", 1), # stack is now [20,i,20,i]
    ("POP", 2), # stack is now [20,i] r0=20 r1=i
    ("ALU", "LT"), # if 20 < i push 1 on stack otherwise 0, stack is now [2,i,2<i]
    ("POP", 1),  # stack is now [20,i] and the result goes to r0
    ("JNZ", "END"), # based on r0 we jump to the end if 20<i otherwise continue to next line
    ("PUSH", 1),  # Next three lines is i++
    ("POP", 2),
    ("ALU", "ADD"),
    ("JMP", "START1"), # return to start1 where we check the condition
    "END:",
    ("POP", 2) # r0=20 r1=
]

program = [
    ("PUSH", 10),
    ("PUSH", 0),
    ("POP", 2),
    ("STOR", 0),
    ("PUSH", 0),
    ("POP", 1),
    ("LOAD" , 0)
]


stack = Array("stack", IntSort(), IntSort())
memory = Array("memory", IntSort(), IntSort())
sp, r0, r1 = Ints('sp r0 r1')

pre_condition = And(sp == 0, stack[0]==0)

chcs_correct = create_chcs(pre_condition=pre_condition,
                           input_vars=[],
                           program=program,
                           post_condition=And(stack[0] == 0, stack[1] == 10, sp == 1, memory[0] == 10))
d = {'xform.inline_eager': False, 'xform.inline_linear': False}
chcs_correct.solve(d)


version 7


U7,¬(ν0[0] ≥ 11) ∧ ¬(ν2 ≤ 0) ∧ ¬(ν1[0] ≥ 1) ∧ ¬(ν1[1] ≥ 11) ∧ ¬(ν1[0] ≤ -1) ∧ ¬(ν0[0] ≤ 9) ∧ ¬(ν1[1] ≤ 9) ∧ ¬(ν2 ≥ 2)
U6,¬(ν1[0] ≤ -1) ∧ ¬(ν0[ν3] ≤ 9) ∧ ¬(ν1[0] ≥ 1) ∧ ¬(ν0[0] ≥ 11) ∧ ¬(ν0[0] ≤ 9) ∧ ¬(ν0[ν3] ≥ 11) ∧ ¬(ν2 ≤ -1) ∧ ¬(ν2 ≥ 1)
U2,¬(ν1[-1 + ν2] ≤ 9) ∧ ¬(ν1[-1 + ν2] ≥ 11) ∧ ¬(ν1[0] ≤ -1) ∧ ¬(ν1[ν2] ≥ 1) ∧ ¬(ν2 ≤ 1) ∧ ¬(ν2 ≥ 3) ∧ ¬(ν1[0] ≥ 1) ∧ ¬(ν1[ν2] ≤ -1)
U4,¬(ν2 ≥ 1) ∧ ¬(ν1[0] ≥ 1) ∧ ¬(ν0[0] ≥ 11) ∧ ¬(ν2 ≤ -1) ∧ ¬(ν1[0] ≤ -1) ∧ ¬(ν0[0] ≤ 9)
U0,¬(ν2 ≥ 1) ∧ ¬(ν1[0] ≥ 1) ∧ ¬(ν1[0] ≤ -1) ∧ ¬(ν2 ≤ -1)
U5,¬(ν2 ≤ 0) ∧ ¬(ν0[ν1[ν2]] ≥ 11) ∧ ¬(ν1[0] ≤ -1) ∧ ¬(ν0[0] ≥ 11) ∧ ¬(ν0[ν1[ν2]] ≤ 9) ∧ ¬(ν2 ≥ 2) ∧ ¬(ν1[0] ≥ 1) ∧ ¬(ν0[0] ≤ 9)
U1,¬(ν1[1] ≥ 11) ∧ ¬(ν2 ≤ 0) ∧ ¬(ν1[1] ≤ 9) ∧ ¬(ν2 ≥ 2) ∧ ¬(ν1[0] ≤ -1) ∧ ¬(ν1[0] ≥ 1)
U3,¬(ν2 ≤ -1) ∧ ν2 = ν4 ∧ ¬(ν3 ≤ 9) ∧ ¬(ν1[0] ≥ 1) ∧ ¬(ν1[0] ≤ -1) ∧ ¬(ν3 ≥ 11) ∧ ¬(ν2 ≥ 1)
